In [1]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import CustomJS, Slider, Range1d, Span, Toggle, Spinner
from bokeh.layouts import column, row
output_notebook()

Loading BokehJS ...

In [2]:
colors = ['#000000', '#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7']
SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

def set_fig_style(fig,xmax,ymax,xlab="x/l₀",ylab="q(x,t)",xmin=0,ymin=None):
    if ymin is None:
        ymin = -ymax
    for (ax, lab) in [(fig.xaxis,xlab),(fig.yaxis,ylab)]:
        ax.axis_label = lab
        ax.axis_label_text_font_size = '14pt'
        ax.axis_label_text_font_style = 'normal'
        ax.major_label_text_font_size = '12pt'
    fig.x_range = Range1d(xmin,xmax)
    fig.y_range = Range1d(ymin,ymax)

# Transversal modes of an infinite chain of oscillators
We here implement and animate the solutions to the normal modes of the transverse oscillations of an infinite chain of masses connected by springs. The normal modes are $e^{i (k x - \omega(k) t)}$ with frequency $\omega(k) = \omega_c \left|\sin \frac{k l_0}{2}\right|$, with $k \in \left(-\frac{\pi}{l_0},\frac{\pi}{l_0}\right]$ and $\omega_c = 2\sqrt{\frac{k_s}{m}\left(1-\frac{l_n}{l_0}\right)}$. Here, $k_s$ is the spring constant, $l_n$ is their natural length, and $l_0$ is the distance between the masses (the tension in the chain is $k_s (l_0 - l_n)$.) Below, we assume that $\omega_c = 1$, which is equivalent to choosing the unit of time as $t_c = 1/\omega_c$ (i.e., the period of oscillation for frequency $\omega_c$ is $2\pi$ time units). We could also use $l_0$ as the unit of distance, such that $l_0=1$. While we set it to this value globally, we will keep $l_0$ in the formulas for clarity.

In [3]:
l0 = 1.
# for most plots, use x=[0,25] as the plotting range
xmax = 25

Plot dispersion relation $\omega(k)$ and phase velocity $v_f = \frac{\omega(k)}{k}$.

In [4]:
# plot positive and negative parts separately to avoid undefined vf at k=0
kps = np.linspace(1e-14,np.pi/l0,100)
ωs = np.abs(np.sin(kps*l0/2))
fig = figure(width=600,height=300)
set_fig_style(fig,xmin=-1,xmax=1,ymax=1.03,xlab='k [π/l₀]',ylab='')
for ks in kps,-kps:
    fig.line(ks/(np.pi/l0),ωs,line_width=4,color=colors[0],legend_label='ω(k)')
    fig.line(ks/(np.pi/l0),ωs/ks,line_width=3,color=colors[1],legend_label='vf(k)')
fig.legend.location = 'bottom_right'
show(fig)

Show the oscillators along a chain (black dots), the springs (black lines), and the underlying mode function (orange line). The slider for $k$ controls the wave number within the range $(-\pi/l_0,\pi/l_0]$. The vertical dashed orange lines show the position of the maxima of the wave. Note that the range of the slider for $t$ automatically changes so that it ranges from $0$ to $T = \frac{2\pi}{\omega(k)}$, i.e., covers exactly one period of oscillation. Press the `► Play` button to animate the wave propagation.

Note that each mass only oscillates up and down, but the overall wave propagates to the left or right depending on the sign of $k$.

In [11]:
def make_mode_animation():
    xm = np.arange(0,xmax+1e-8,l0)
    x  = np.arange(0,xmax+1e-8,l0/20)

    krel0 = 0.2
    k0 = krel0*np.pi/l0
    source_x = ColumnDataSource(data=dict(x=x, y=np.cos(k0*x)))
    source_xm = ColumnDataSource(data=dict(x=xm, y=np.cos(k0*xm)))

    fig = figure(width=900,height=500)
    fig.line('x','y',source=source_x,line_width=2,color=colors[1])
    fig.line('x','y',source=source_xm,line_width=0.5,color=colors[0])
    fig.circle('x','y',source=source_xm,size=6,color=colors[0])
    # max number of maxima: xmax / λmin = xmax / (2π/kmax) = xmax / (2π/(π/l₀)) = xmax / (2 l₀)
    sps = [Span(location=ii*2*np.pi/k0, dimension='height', 
                line_color=colors[1], line_width=1.5,line_dash=[6,4]) for ii in range(int(xmax/(2*l0))+1)]
    fig.renderers.extend(sps)

    set_fig_style(fig,xmax,1.2)

    slider_k = Slider(start=-1, end=1, value=krel0, step=0.01, title="k [π/l₀]")
    slider_t = Slider(start=0, end=20, value=0, step=0.01, title="t", sizing_mode="stretch_both")

    # JavaScript callback to do animations directly in the browser (without needing to communicate with server)
    cb_sliders = CustomJS(args=dict(sources=[source_x,source_xm], slider_k=slider_k,
                                    slider_t=slider_t, sps=sps, l0=l0),
                          code="""const t = slider_t.value;
                                  const k = slider_k.value*Math.PI/l0;
                                  const omega = Math.abs(Math.sin(k*l0/2));
                                  slider_t.end = omega == 0 ? 100. : 2*Math.PI/omega;
                                  sources.forEach(source => {
                                      const ys = source.data['y'];
                                      source.data['x'].forEach((x,i) => ys[i] = Math.cos(k*x - omega*t));
                                      source.change.emit();
                                  })
                                  const vf = (k==0 ? 0. : omega/k);
                                  const λ = 2*Math.PI/Math.abs(k);
                                  sps.forEach((sp,i) => sp.location = vf*t + (i+(vf<0))*λ)""")
    for slider in [slider_k,slider_t]:
        slider.js_on_change('value', cb_sliders)

    # Set up Play/Pause button/toggle JS
    cb_toggle = CustomJS(args=dict(s_t=slider_t),
                         code="""function upfun() { s_t.value = (s_t.value + 0.1) % s_t.end; cb_obj.active || clearInterval(updater) }
                                 cb_obj.label = cb_obj.active ? '❚❚ Pause' : '► Play';
                                 if (cb_obj.active) var updater = setInterval(upfun, 20);""")
    toggle = Toggle(label='► Play',active=False, sizing_mode="stretch_height", width=80)
    toggle.js_on_change('active', cb_toggle)

    layout = column(slider_k,row(toggle,slider_t),fig)
    
    show(layout)

make_mode_animation()

Show two modes (black and orange lines). Notice, e.g., that waves with smaller $k$ have larger phase velocity $v_f$ since the absolute value of $v_f = \frac{\omega(k)}{k} = \frac{|\sin(k l_0/2)|}{k}$ decreases with $|k|$.

In [12]:
def make_two_mode_animation():
    x = np.arange(0,xmax+1e-8,l0/20)

    krel0s = np.r_[0.1,0.73]
    k0s = krel0s*np.pi/l0
    source = ColumnDataSource(data = dict(x=x) | {f'y{ik}':np.cos(k*x) for ik,k in enumerate(k0s)})

    fig = figure(width=900,height=500)
    spcol = []
    for ik,k0 in enumerate(k0s):
        fig.line('x',f'y{ik}',source=source, line_width=2, color=colors[ik])
        sps = [Span(location=ii*2*np.pi/k0,dimension='height',line_color=colors[ik],line_width=1,line_dash=[6,4]) for ii in range(int(xmax/(2*l0))+1)]
        fig.renderers.extend(sps)
        spcol.append(sps)

    set_fig_style(fig,xmax,1.2)

    sliders_k = [Slider(start=-1, end=1, value=krel0, step=0.01, title="k [π/l₀]") for krel0 in krel0s]
    slider_t = Slider(start=0, end=1000, value=0, step=0.1, title='t', sizing_mode="stretch_both")

    # JavaScript callback to do animations directly in the browser (without needing to communicate with server)
    cb_sliders = CustomJS(args=dict(source=source, slider_t=slider_t, sliders_k=sliders_k, spcol=spcol, l0=l0),
                          code="""const t = slider_t.value;
                                  const xs = source.data['x'];
                                  for (var ik=0; ik<sliders_k.length; ik++) {
                                      var ys = source.data[`y${ik}`];
                                      var k = sliders_k[ik].value*Math.PI/l0;
                                      var omega = Math.abs(Math.sin(k*l0/2));
                                      xs.forEach((x,i) => ys[i] = Math.cos(k*x - omega*t));
                                      if (k != 0) {
                                          const vf = omega/k;
                                          const λ = 2*Math.PI/Math.abs(k);
                                          const tint = t % (2*Math.PI/omega);
                                          spcol[ik].forEach((sp,i) => sp.location = vf*tint + (i+(vf<0))*λ)
                                       } else {
                                          spcol[ik].forEach(sp => sp.location = 0)
                                       }
                                   }
                                   source.change.emit();""")
    for slider in sliders_k + [slider_t]:
        slider.js_on_change('value', cb_sliders)

    # Set up Play/Pause button/toggle JS
    cb_toggle = CustomJS(args=dict(s_t=slider_t),
                         code="""function upfun() { s_t.value = (s_t.value + 0.1) % s_t.end; cb_obj.active || clearInterval(updater) }
                                 cb_obj.label = cb_obj.active ? '❚❚ Pause' : '► Play';
                                 if (cb_obj.active) var updater = setInterval(upfun, 20);""")
    toggle = Toggle(label='► Play',active=False, sizing_mode="stretch_height", width=80)
    toggle.js_on_change('active', cb_toggle)

    layout = column(*sliders_k,row(toggle,slider_t),fig)
    
    show(layout)

make_two_mode_animation()

Plot a solution consisting of $n$ normal modes. To change from 2 to more modes, add additional entries to the list used for calling the function `make_n_mode_sum_animation`. The list contains the initial values for the sliders $k_i$. The sum over all normal modes is shown in black, with each contributing mode shown as a semi-transparent line. Change the value of the `δt` spinner to change the speed of the animation, and use the `xmax` slider to change the range of values $x$ that is plotted.

Some suggestions for values of the different ks: $[0.3,0.32]$, $[0.2,0.4]$, $[0.1,0.2,0.3]$.

In [15]:
def make_n_mode_sum_animation(krel0s,cs=None,xmax=200):
    krel0s = np.asarray(krel0s)
    cs = np.ones_like(krel0s) if cs is None else np.asarray(cs)
    assert krel0s.ndim == 1
    assert krel0s.shape == cs.shape

    k0s = krel0s*np.pi/l0
    x = np.linspace(0,xmax+1e-8,2000)
    source = ColumnDataSource(data = dict(x=x) | {f'y{ik}':np.cos(k*x) for ik,k in enumerate(k0s)})
    source.data['y'] = np.sum([source.data[f'y{ik}'] for ik,k in enumerate(k0s)],axis=0)

    fig = figure(width=900,height=500)
    fig.line('x','y',source=source, line_width=4, color=colors[0])
    for ik,k0 in enumerate(k0s):
        fig.line('x',f'y{ik}',source=source, line_width=2, line_alpha=0.5, color=colors[(ik+1)%len(colors)])

    set_fig_style(fig,xmax,1.2)
    del fig.y_range

    sliders_k = [Slider(start=-1, end=1, value=krel0, step=0.01, title=f"k{ik+1} [π/l₀]".translate(SUB)) for ik,krel0 in enumerate(krel0s)]
    sliders_c = [Slider(start=0,  end=1, value=1,     step=0.01, title=f"c{ik+1}".translate(SUB))        for ik,krel0 in enumerate(krel0s)]
    slider_t = Slider(start=0, end=1000, value=0, step=0.1, title='t', sizing_mode="stretch_both")
    slider_xmax = Slider(start=25, end=300, value=xmax, step=1, title='xmax', sizing_mode="stretch_both")
    spinner_dt = Spinner(low=0.05, high=5., step=0.05, value=0.1, title="δt", width=80, sizing_mode="stretch_height")

    # JavaScript callback to do animations directly in the browser (without needing to communicate with server)
    cb_sliders = CustomJS(args=dict(source=source, slider_t=slider_t, slider_xmax=slider_xmax,
                                    sliders_k=sliders_k, sliders_c=sliders_c, l0=l0, fig=fig),
                          code="""const t = slider_t.value;
                                  const xmax = slider_xmax.value;
                                  fig.x_range.end = xmax;
                                  const xs = source.data['x'];
                                  const ys = source.data['y'];
                                  const nx = xs.length;
                                  xs.forEach((_,i) => { xs[i] = i*xmax/(nx-1); ys[i] = 0 });
                                  for (var ik=0; ik<sliders_k.length; ik++) {
                                      var yk = source.data[`y${ik}`];
                                      var k = sliders_k[ik].value*Math.PI/l0;
                                      var c = sliders_c[ik].value;
                                      var omega = Math.abs(Math.sin(k*l0/2));
                                      xs.forEach((x,i) => { yk[i] = c * Math.cos(k*x - omega*t);
                                                            ys[i] += yk[i]; })
                                  }
                                  source.change.emit();""")
    for slider in sliders_k + sliders_c + [slider_t,slider_xmax]:
        slider.js_on_change('value', cb_sliders)

    # Set up Play/Pause button/toggle JS
    cb_toggle = CustomJS(args=dict(s_t=slider_t,s_dt=spinner_dt),
                         code="""function upfun() { s_t.value += s_dt.value; cb_obj.active || clearInterval(updater) }
                                 cb_obj.label = cb_obj.active ? '❚❚ Pause' : '► Play';
                                 if (cb_obj.active) var updater = setInterval(upfun, 20);""")
    toggle = Toggle(label='► Play',active=False, sizing_mode="stretch_height", width=80)
    toggle.js_on_change('active', cb_toggle)

    layout = column(*[row(s_k,s_c,sizing_mode="stretch_both") for (s_k,s_c) in zip(sliders_k,sliders_c)],
                    row(toggle, spinner_dt, slider_t),
                    slider_xmax,
                    fig)
    show(layout)

make_n_mode_sum_animation([0.3,0.32])

Same as above, but do not create sliders for the wave numbers or amplitudes, just use the values passed directly. This is useful to use a superposition of many modes without having 100 sliders, and also allows complex mode amplitudes without needing two sliders. You can also choose to show the separate normal modes by passing `show_modes` (`=True` or `=False`), and show the individual point masses moving with `show_masses` (`=True` or `=False`).

In [19]:
def make_n_mode_sum_animation_nosliders(krels,cs,xmax=200,show_modes=True,show_masses=False,dx=None):
    krels = np.asarray(krels)
    cs = np.asarray(cs)
    assert krels.ndim == 1
    assert krels.shape == cs.shape

    acs = abs(cs)
    phis = np.angle(cs)

    def make_source(x,include_modes):
        source = ColumnDataSource(data = dict(x=x,y=0*x))
        for ik,(k,ac,phi) in enumerate(zip(ks,acs,phis)):
            yi = ac*np.cos(k*x+phi)
            source.data['y'] += yi
            if include_modes:
                source.data[f'y{ik}'] = yi
        return source
    
    ks = krels*np.pi/l0
    if dx is None:
        # resolution: take 40 points per shortest wavelength in the packet
        # dx = λmin / 40 = 2π/kmax / 40
        dx = 2*np.pi/(40*ks.max())
    
    sources = [make_source(np.arange(0,xmax,dx), show_modes)]
    if show_masses:
        sources.append(make_source(np.arange(0,xmax,l0), False))
            
    fig = figure(width=900,height=500)
    fig.line('x','y',source=sources[0], line_width=4, color=colors[0])
    if show_modes:
        for ik,k in enumerate(ks):
            fig.line('x',f'y{ik}',source=sources[0], line_width=2, line_alpha=0.5,
                     color=colors[1 + (ik%(len(colors)-1))])
    if show_masses:
        fig.circle('x','y',source=sources[1],size=6,color=colors[0])
            
    set_fig_style(fig,xmax,ymax=1.03*abs(sources[0].data['y']).max())

    slider_t = Slider(start=0, end=1000, value=0, step=0.1, title='t', sizing_mode="stretch_both")
    spinner_dt = Spinner(low=0.05, high=5., step=0.05, value=0.1, title="δt", width=80, sizing_mode="stretch_height")

    # JavaScript callback to do animations directly in the browser (without needing to communicate with server)
    cb_sliders = CustomJS(args=dict(sources=sources, slider_t=slider_t, ks=ks, acs=acs,
                                    phis=phis, l0=l0, fig=fig, show_modes=show_modes),
                          code="""const t = slider_t.value;
                                  const omegas = ks.map(k => Math.abs(Math.sin(k*l0/2)));
                                  sources.forEach((source,isource) => {
                                      const xs = source.data['x'];
                                      const ys = source.data['y'];
                                      const do_modes = show_modes && (isource==0);
                                      ys.forEach((_,i) => ys[i] = 0);
                                      for (var ik=0; ik<ks.length; ik++) {
                                          if (do_modes) { var yk = source.data[`y${ik}`] };
                                          const k = ks[ik], omega = omegas[ik], ac = acs[ik], phi = phis[ik];
                                          xs.forEach((x,i) => { const ym = ac * Math.cos(k*x - omega*t + phi);
                                                                ys[i] += ym;
                                                                if (do_modes) yk[i] = ym; })
                                      }
                                      source.change.emit()
                                  })""")
    slider_t.js_on_change('value', cb_sliders)

    # Set up Play/Pause button/toggle JS
    cb_toggle = CustomJS(args=dict(s_t=slider_t,s_dt=spinner_dt),
                         code="""function upfun() { s_t.value += s_dt.value; cb_obj.active || clearInterval(updater) }
                                 cb_obj.label = cb_obj.active ? '❚❚ Pause' : '► Play';
                                 if (cb_obj.active) var updater = setInterval(upfun, 20);""")
    toggle = Toggle(label='► Play',active=False, sizing_mode="stretch_height", width=80)
    toggle.js_on_change('active', cb_toggle)

    layout = column(row(toggle, spinner_dt, slider_t), fig)
    show(layout)

make_n_mode_sum_animation_nosliders([0.3,0.32,0.34],[0.5,1.,0.5])

Use the code above to create a wavepacket with many components (30 values ranging from $k=0$ to $k=0.3 \pi/l_0$), with a Gaussian distribution of amplitudes and a complex phase corresponding to a position offset. Notice that during the propagation (set $\delta t$ to larger values to see it better), the wavepackets slowly change shape. This is because the dispersion relation is not perfectly linear.

In [20]:
krels = np.linspace(0.0,0.3,30)
cs = np.exp(-(krels-0.15)**2/(2*0.03**2) - 1j*40*krels*np.pi/l0)

fig = figure(height=180,width=900)
fig.line(krels,abs(cs),line_width=1,color=colors[0])
fig.circle(krels,abs(cs),size=7,color=colors[0])
set_fig_style(fig,krels[-1]*1.01,1.)
del fig.y_range
fig.xaxis.axis_label = "k [π/l₀]"
fig.yaxis.axis_label = "|c(k)|"
show(fig)

make_n_mode_sum_animation_nosliders(krels,cs,xmax=300,show_modes=False)

## create solutions for finite chains by choosing solutions of the infinite one that satisfy the boundary conditions
For example, we assume that the masses at positions $x=0$ and $x=L$ (where $L = m l_0$, with $m>1$ an integer) are fixed at $y=0$, i.e., we have the boundary condition $y(x=0,t) = y(x=L,t) = 0$.

We also note that due to the degeneracy $\omega(k)=\omega(-k)$, any sum of plane waves with $k$ and $-k$ is also a normal mode, $c_+ e^{i(kx-\omega t} + c_- e^{i(-kx-\omega t)} = (c_+ e^{ikx} + c_- e^{-ikx}) e^{-i\omega t}$. In particular, the choices $c_- = \pm c_+$ give standing waves, as $e^{i k x} + e^{-i k x} = 2 \cos(k x)$ and $e^{i k x} - e^{-i k x} = 2i \sin(k x)$. We can thus use standing cosine and sine waves instead of propagating plane waves as an equally valid normal mode basis. In order to fulfill $y(x=0,t) = 0$, the cosine-modes cannot contribute. Furthermore, in order to fulfill $y(x=L,t)=0$, we need that $\sin(k L) = 0$, i.e., only the discrete values $k_n =\frac{n\pi}{L}$, with $n \in \mathbb{N}^+$ are allowed.

Note that the normalized `krel` below is $k \frac{l_0}{\pi}$, so $k_n =\frac{n\pi}{L}$ corresponds to $k_{\mathrm{rel}} = \frac{n l_0}{L}$. Since $k_{\mathrm{rel}} \in (-1,1]$, $n$ is restricted to a maximum value of $N = \frac{L}{l_0}$. However, the masses for $n=N$ are exactly at nodes of the sine wave ($\sin \frac{L}{l_0} \frac{\pi}{L} x = \sin j \pi = 0$ for $x=j l_0$), so this mode describes no motion. The highest actual value of $n$ is thus $n_\max = N-1$. Note that the number of discrete modes that appears is thus exactly the number of free masses, as it should be. Finally, since this is just an alternative way to arrive at the same solution, the frequencies $\omega_n = \omega_c |\sin(k_n l_0/2)|$ are exactly the same as those obtained by diagonalizing the matrices $\hat{M}$ and $\hat{K}$ describing the same system, and the eigenvectors (either obtained by evaluating the mode functions at $x=jl_0$ or by diagonalizing the matrices) naturally also agree.

In [23]:
L = 5*l0
nmax = int(L/l0 - 0.5)
print("Number of normal modes:", nmax)
krpos = np.arange(1,nmax+1) * l0/L
cpos = np.r_[1.3,0.4,0.2,np.zeros(nmax-3)]
krels = np.r_[-krpos[::-1],krpos]
# make the complex coefficients -i c_n and i c_n to get real sine waves
cs = -0.5j * np.r_[-cpos[::-1],cpos]

# only pass the actually nonzero modes
inds = np.nonzero(cs)
krels = krels[inds]
cs = cs[inds]

make_n_mode_sum_animation_nosliders(krels,cs,xmax=L+1e-8,show_modes=True,show_masses=True)

Number of normal modes: 4


In [24]:
L = 800*l0
nmax = int(L/l0 - 0.5)
print("Number of normal modes:", nmax)
ns = np.arange(1,nmax+1)
krpos = ns * l0/L
cpos = 1/ns**2 * np.sin(ns*np.pi/2)
# only pass the actually nonzero modes
inds, = np.where(abs(cpos)>1e-7)
krpos = krpos[inds]
cpos = cpos[inds]
print("Number of modes with non-zero amplitude:", len(cpos))

krels = np.r_[-krpos[::-1],krpos]
# make the complex coefficients -i c_n and i c_n to get real sine waves
cs = -0.5j * np.r_[-cpos[::-1],cpos]

make_n_mode_sum_animation_nosliders(krels,cs,xmax=L+1e-8,show_modes=False,show_masses=False,dx=l0)

Number of normal modes: 799
Number of modes with non-zero amplitude: 400
